# 1. include class info in the json result

In [ ]:
from dataclasses import dataclass,field
from typing import Set, Optional

from dataclasses_json import dataclass_json,global_config


@dataclass_json
@dataclass
class Animal:
    id: int = 0
    health: int = 100


@dataclass_json
@dataclass
class Cat(Animal):
    age: int = 1

@dataclass_json
@dataclass
class Dog(Animal):
    age: int = 1

@dataclass_json
@dataclass
class PetCat(Cat):
    name: str = ''

@dataclass_json
@dataclass
class Person:
    name:str = 'zyx'
    animals: list[Animal] = field(default_factory=lambda:[])


In [2]:
p1=Person(animals=[Animal(),Cat(),PetCat()])
p1.to_dict()

{'name': 'zyx',
 'animals': [{'id': 0, 'health': 100},
  {'id': 0, 'health': 100, 'age': 1},
  {'id': 0, 'health': 100, 'age': 1, 'name': ''}]}

In [3]:
p2 = Person.from_dict(p1.to_dict())
p2.to_dict()

{'name': 'zyx',
 'animals': [{'id': 0, 'health': 100},
  {'id': 0, 'health': 100},
  {'id': 0, 'health': 100}]}

some fields are missing!

to solve this, we need to include class info into the result

In [5]:
global_config.include_class_info=True
p1.to_dict()

{'__module__': '__main__',
 '__name__': 'Person',
 'name': 'zyx',
 'animals': [{'__module__': '__main__',
   '__name__': 'Animal',
   'id': 0,
   'health': 100},
  {'__module__': '__main__',
   '__name__': 'Cat',
   'id': 0,
   'health': 100,
   'age': 1},
  {'__module__': '__main__',
   '__name__': 'PetCat',
   'id': 0,
   'health': 100,
   'age': 1,
   'name': ''}]}

In [7]:
p2 = Person.from_dict(p1.to_dict())
global_config.include_class_info=False
p2.to_dict()

{'name': 'zyx',
 'animals': [{'id': 0, 'health': 100},
  {'id': 0, 'health': 100, 'age': 1},
  {'id': 0, 'health': 100, 'age': 1, 'name': ''}]}

now the fields are all restored!

# 2. use cache to save time

if i have thousands of objects to jsonize, the code will waste much time on get dataclass info, which will not change however in the process of jsonization 

In [13]:
import cProfile
import pstats
global_config.enable_cache=False
p3 = Person(animals=[Animal() for _ in range(100000)])

# Create a cProfile object
pr = cProfile.Profile()

# Enable profiling
pr.enable()

# Run your code
result_without_cache = p3.to_json()

# Disable profiling
pr.disable()

# Save the stats to a file
pr.dump_stats('profile_stats')

# Open the saved stats file in Jupyter Notebook
stats = pstats.Stats('profile_stats')
stats.sort_stats('cumulative')
stats.print_stats()

Wed Feb 28 21:25:23 2024    profile_stats

         11802740 function calls (10302726 primitive calls) in 6.541 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    6.541    3.270 c:\Users\ZYX\miniconda3\envs\dsj\Lib\site-packages\IPython\core\interactiveshell.py:3541(run_code)
        2    0.000    0.000    6.541    3.270 {built-in method builtins.exec}
        1    0.000    0.000    6.541    6.541 C:\Users\ZYX\AppData\Local\Temp\ipykernel_21292\461415860.py:1(<module>)
        1    0.004    0.004    6.541    6.541 d:\workspace\dataclasses-json\dataclasses_json\api.py:26(to_json)
        1    0.000    0.000    6.488    6.488 d:\workspace\dataclasses-json\dataclasses_json\api.py:72(to_dict)
 300003/1    0.772    0.000    6.488    6.488 d:\workspace\dataclasses-json\dataclasses_json\core.py:429(_asdict)
   100001    0.057    0.000    6.462    0.000 d:\workspace\dataclasses-json\dataclasses_json\co

In [18]:
import cProfile
import pstats
global_config.enable_cache=True
p3 = Person(animals=[Animal() for _ in range(100000)])

# Create a cProfile object
pr = cProfile.Profile()

# Enable profiling
pr.enable()

# Run your code
result_with_cache = p3.to_json()

# Disable profiling
pr.disable()

# Save the stats to a file
pr.dump_stats('profile_stats')

# Open the saved stats file in Jupyter Notebook
stats = pstats.Stats('profile_stats')
stats.sort_stats('cumulative')
stats.print_stats()

Wed Feb 28 21:26:30 2024    profile_stats

         5702755 function calls (4602444 primitive calls) in 2.713 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100001    0.041    0.000    2.664    0.000 d:\workspace\dataclasses-json\dataclasses_json\core.py:459(<genexpr>)
1500355/700052    0.493    0.000    1.168    0.000 {built-in method builtins.isinstance}
400006/400004    0.158    0.000    0.783    0.000 c:\Users\ZYX\miniconda3\envs\dsj\Lib\typing.py:1175(__instancecheck__)
400006/400005    0.191    0.000    0.625    0.000 c:\Users\ZYX\miniconda3\envs\dsj\Lib\typing.py:1446(__subclasscheck__)
400006/400005    0.141    0.000    0.327    0.000 {built-in method builtins.issubclass}
   300003    0.147    0.000    0.241    0.000 c:\Users\ZYX\miniconda3\envs\dsj\Lib\dataclasses.py:1292(is_dataclass)
   100001    0.066    0.000    0.208    0.000 d:\workspace\dataclasses-json\dataclasses_json\utils.py:203(_handle_undefined_p

The improvement in program speed is huge, from 6.6s to 2.5s in my laptop

now let's check if the results are the same

In [19]:
result_with_cache==result_without_cache

True